## Determine shortest distance to nearest Surface in Imaris, v1.0

### Author: Dr David Elliott, Hotchkiss Brain Institute - Advanced Microscopy Platform

### User: Dr Rajiv Jain (Yong lab)

### Date: 19th of May, 2022

### GOAL:
Imaris is able to calculate distance to nearest neighbours for spots, however, it does not have this functionality for Surfaces. It does however provide us with the distance to Surfaces that belong to a different object (ie., a surface that was generated spearately, as opposed to surfaces within the same object) which it referes to as 'surface-to-surface'.

Thus, a work-around is to convert all surfaces within an object to a separate Surface object, using the 'split surfaces' XTension inbuilt to Imaris. Now, each surface/cell will be its own object, and these will all be sorted into a new folder. While these objects are in this folder, surface-to-surface calculations will not be provided. However, if we now select and drag all of these objects out of the folder and into the main menu, surface-to-surface shortest distances measurements will be provided.

If we now go back to our original surfaces object (which contains many surfaces), we can get a csv file for distance to every newly created object/surface. The obstacle is that there are so many (typically in the 000's) and each surface in the original object will have a correspnding object, which will have a distance of 0.

The User will export all stats from the original surfaces object as csv into one folder. This script will then ask for the location of this folder and the name of the LAST shortest-distance csv file. It will then calculate the total number of files, open each one, sort by shortest distance to find the 'second' nearest neighbour, which is the 'real' nearest neighbour, then export these values to 'summary.csv'


In [ ]:
# Import modules

import numpy as np
import pandas as pd
import csv
import os.path
from os import path

In [215]:
# Prompt User for location of folder containing results and the last file. 
# NOTE: this can be entered directly in 'Option 2' below. Just comment out and uncomment sections accordingly. Prompting the User ensures that the
# folder and filenames get re-entered every time for each dataset (thus reducing the probablility of making an error when working with many image datasets).

def checkFolder():
    
    valid = False
    while (valid == False):
        
        enteredFolder = input("Please enter the location of the folder containing the 'Shortest_Distance_to_Surfaces...etc' files: ")
        
        if path.exists(enteredFolder) == False:  # Confirm that a valid directory was entered
            print ("\nSorry, was unable to locate that folder. Please try again (go to Kernel/Restart Kernel to get out of this loop).\n")
        else:
            return(enteredFolder)
            
                
def checkFile():
    
    valid = False
    while (valid == False):
        
        enteredFilename = input("Please enter the fliename for the LAST 'Shortest_Distance_to_Surfaces...etc' file ('.csv' will be added automatically): ")
        
        try:
            with open((folder + '\\' + enteredFilename + '.csv'), newline = "\n") as csvfile:
                return (enteredFilename + '.csv')
            
        except:
            print ("\nSorry, was unable to locate that folder. Please try again (go to Kernel/Restart Kernel to get out of this loop).\n")
        

# Location of folder containing the Shortest-distance csv files        
folder = checkFolder()

# Filename for the LAST 'Shortest_Distance_to_Surfaces...etc' file ('.csv' will be added automatically)
lastFile = checkFile()


# Option 2: Uncomment below
# # Location of folder containing csv files
# folder = (r'C:\Users\david\Desktop\Rajiv\Intra-object_distances\Stats_test_3\SC4 L1_David_Statistics')

# # Name of the LAST file in the 'Shortest_Distance_to_Surfaces...etc' file series
# lastFile = (r'SC4 L1_David_Shortest_Distance_to_Surfaces_Surfaces=Neutrophils_[1163]') + '.csv'

Please enter the location of the folder containing the 'Shortest_Distance_to_Surfaces...etc' files:  D:\Myeloid analysis exp 2\Same cell interaction exports\Neutrophils\SC7 L2_Statistics
Please enter the fliename for the LAST 'Shortest_Distance_to_Surfaces...etc' file ('.csv' will be added automatically):  SC7 L2_Shortest_Distance_to_Surfaces_Surfaces=Neutrophils_[22]


In [216]:
# Determine the number of the last file [in brackets]

def findLocation(file):    
    
    counter = -6
    found = False

    while found == False:

        counter = (counter - 1)
        #print(counter)
        if file[counter] == '[':
            found = True

    return(counter + 1)


# Using the last filename, parse out the number of the last file which is between the '[]'. Example, 1163 in the following:
# SC4 L1_David_Shortest_Distance_to_Surfaces_Surfaces=Neutrophils_[1163].csv

# We know the length from the end of the file before we reach the number ('].csv') is consistent, thus we can start moving 
# backward until we reach the '['. This will be performed by the function 'findLocation'. This positional information
# will then enable us to extract the number in between and store in the variable 'numberOfObjects'. 
startLocation = findLocation(lastFile)

numberOfObjects = lastFile[int(startLocation):-5]

# Now extract the common prefix for all the files (ie., before the numbering) and store in 'filePrefix'
filePrefix = lastFile[:-5-len(numberOfObjects)]



In [217]:

def findNearest(ID):
    '''
    Receive: Cell/file ID number
    Return: The real distance to nearest neighbour
    
    - ID number (for the file, which is +1 relative to the Cell ID#) is used to determine which file to open. 
    This file is read into a df.
    - The df is sorted by "Shortest Distance to Surfaces" values and the index is rest.
    - Now, the second value in the "Shortest Distance to Surfaces" column (loc 1, as counting starts from 0),
    reflects the real nearest neighbour. This value is returned.
    '''
    
    # Open csv file
    tempDf = pd.read_csv((folder + '\\' + filePrefix + str(ID) + '].csv'), header = 2)
    
    # Sort by shortest Distance values
    sortedDf = tempDf.sort_values(["Shortest Distance to Surfaces"], ascending = True)
    
    # Reset index
    sortedDf = sortedDf.reset_index(drop = True)
    
    # Return nearest neighhbour value (second value in index)
    return(sortedDf.loc[1, "Shortest Distance to Surfaces"])
    


In [218]:
# Initialise df 'summaryDf' to store results
summaryDf = pd.DataFrame(columns = ["Cell ID", "Nearest neighbour (um)"])


# Loop through all of the files, determine value to nearest neighbour (ie., second smallest value), 
# then write to df alongside Cell ID#
for i in range(int(numberOfObjects)):
    summaryDf.loc[i, "Cell ID"] = i
    summaryDf.loc[i, "Nearest neighbour (um)"] = findNearest(i+1)

    
# Write to a csv file. By default this will be saved to the folder location provided.
summaryDf.to_csv(folder + '\\' + 'Summary.csv')
print("FINISHED!")

FINISHED!
